Implementieren Sie selbst das Otsu-Verfahren (naive oder effiziente Implementierung,
das dürfen Sie entscheiden) und testen, dass es für die Binarisierung von Grauwertbildern
korrekte Ergebnisse liefert. Vergleichen Sie Ihre Ergebnisse mit denen der OpenCv-
Implementierung (cv.threshold(img, ..., cv.THRESH_BINARY+cv.THRESH_OTSU))

In [2]:
import cv2
import numpy as np

# Bild I einlesen
I = cv2.imread("/Users/aloysius/Library/Mobile Documents/com~apple~CloudDocs/TH Köln/12. Semester/IBV/Übungen/Übung 3/Lena.jpg", cv2.IMREAD_GRAYSCALE)
color_image = cv2.imread("/Users/aloysius/Library/Mobile Documents/com~apple~CloudDocs/TH Köln/12. Semester/IBV/Übungen/Übung 3/Lena.jpg")

hist = cv2.calcHist([I], [0], None, [256], [0, 256])
hist = hist.flatten()  # Convert to 1D array



# Implementation of Otsu's method
def otsu_naive(hist):
    pixel_count = sum(hist)
    tMax = 0
    varMax = -1
    
    for t in range(1, 256):
        # Reset accumulators for each threshold t
        c0, c1 = 0, 0
        m0, m1 = 0, 0
        
        # Calculate class probabilities and means
        for i in range(t):
            c0 += hist[i]
            m0 += i * hist[i]
            
        for i in range(t, 256):
            c1 += hist[i]
            m1 += i * hist[i]
        
        # Avoid division by zero
        if c0 == 0 or c1 == 0:
            continue
            
        # Calculate means
        m0 = m0 / c0 if c0 > 0 else 0
        m1 = m1 / c1 if c1 > 0 else 0
        
        # Calculate weights
        w0 = c0 / pixel_count
        w1 = c1 / pixel_count
        
        # Calculate between-class variance
        var = w0 * w1 * (m0 - m1) ** 2
        
        # Update threshold if variance is higher
        if var > varMax:
            varMax = var
            tMax = t
    
    return tMax

            
# Show the original image
cv2.imshow('Original Image', I)

# Calculate Otsu's threshold
threshold = otsu_naive(hist)

# Apply thresholding
ret, binary_image = cv2.threshold(I, threshold, 255, cv2.THRESH_BINARY)
cv2.imshow('Binary Image (Naive Otsu)', binary_image)
cv2.imwrite('otsu_custom.png', binary_image)

# Otsu's thresholding using OpenCV
ret, thr = cv2.threshold(I, 0, 255, cv2.THRESH_OTSU)
cv2.imshow('Binary Image (OpenCV Otsu)', thr)
cv2.imwrite('otsu_opencv.png', thr)

# Adaptive thresholding
th3 = cv2.adaptiveThreshold(I,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
cv2.imshow('Adaptive Threshold', th3)
cv2.imwrite('adaptive_thresh.png', th3)


# Morphological operations
kernel1 = np.ones((1, 1), np.uint8)
kernel2 = np.ones((2, 2), np.uint8)
kernel3 = np.ones((3, 3), np.uint8)

dilated1 = cv2.dilate(binary_image, kernel1, iterations=1)
dilated2 = cv2.dilate(binary_image, kernel2, iterations=1)
dilated3 = cv2.dilate(binary_image, kernel3, iterations=1)

# Show the dilated image
cv2.imshow('Dilated Image', dilated1)
cv2.imwrite('dilated_image.png', dilated1)
# cv2.imshow('Dilated Image 2', dilated2)
# cv2.imshow('Dilated Image 3', dilated3)


# Erosion, masking original image with binary image
masked_image = cv2.bitwise_or(color_image, color_image, mask=binary_image)
cv2.imshow('Masked Image', masked_image)
cv2.imwrite('masked_image.png', masked_image)

# Wait for a key press and close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1